In [1]:
import os
%pwd

'd:\\End to End ML Projects\\End-to-End-ML-Projects\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\End to End ML Projects\\End-to-End-ML-Projects'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: int

In [4]:
from bikeSharing.constants import *
from bikeSharing.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
from bikeSharing import logger
from sklearn.linear_model import LinearRegression  
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        regressor = LinearRegression().fit(train_x, train_y) 

        joblib.dump(regressor, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-12-12 11:31:53,015: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-12 11:31:53,017: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-12 11:31:53,018: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-12 11:31:53,019: INFO: common: created directory at: artifacts]
[2025-12-12 11:31:53,020: INFO: common: created directory at: artifacts/model_trainer]
